In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
epsilon = 1e-5

# 4 - 朴素贝叶斯

首先要注意的非常重要的一点是，**朴素贝叶斯方法是生成方法**，需要首先根据训练数据学习到联合概率分布$P(X,Y)$，然后需要注意的一点是**朴素贝叶斯的后验概率最大等价于0-1损失函数时的期望风险最小化**

朴素贝叶斯建立在一个条件独立性假设（在相同y值的条件下，x的所有属性相互独立）
$$
\begin{split}
P(X=x\mid Y = c_k) & = P(X^{(1)}=x^{(1)},\cdots, X^{(n)}=x^{(n)} \mid Y=c_k) \\
    & = \prod_{j=1}^nP(X^{(j)}=x^{(j)} \mid Y = c_k)
\end{split}
$$
然后根据贝叶斯公式得到后验概率
$$
\begin{split}
P(Y=c_k\mid X=x) & = \frac{P(X=x\mid Y=c_k)P(Y=c_k)}{\sum_k P(X=x\mid Y=c_k)P(Y=c_k)} \\
    & = \frac{P(Y=c_k)\prod_jP(X^{(j)}=x^{(j)}\mid Y=c_k)}{\sum_k P(Y=c_k)\prod_jP(X^{(j)}=x^{(j)}\mid Y=c_k)}
\end{split}
$$
则
$$
y = \arg\underset{c_k}{\max}P(Y=c_k)\prod_jP(X^{(j)}=x^{(j)}\mid Y=c_k)
$$
我感觉引入贝叶斯并且加上了条件独立性假设的目的是，减少统计时候的参数空间大小，就像书上说的
> 条件概率分布$P(X=x\mid Y=c_k)$有**指数级数量的参数**，其估计实际是不可行的。事实上，假设$x^{(j)}$可取有$S_j$个，$j=1,2,\cdots,n$，$Y$可取值有$K$个，那么参数个数为$K\prod_{j=1}^n S_j$

其实其中所有的有统计直接得到的概率，其实都是由极大似然得到的，只不过都是离散的分布

In [2]:
def naive_Bayes(x, X, Y): ## 这里为了后面好写，把所有的值都统计了
    N, n = X.shape ## N样本数量，n属性个数
    k = np.max(Y) + 1
    Sx = np.max(X, axis=0) + 1
    CX_Y = [] ## 所有条件的计数
    CY = np.zeros(k) ## 所有先验的计数
    
    for i in range(k):
        CY[i] = np.count_nonzero(Y[:,0]==i)
    for i in range(n): ## 每个属性
        x_y = np.zeros((k, Sx[i]))
        for j in range(k): ## 每个y值
            t = X[np.where(Y[:,0]==j)[0]]
            for l in range(Sx[i]):
                x_y[j,l] = np.count_nonzero(t[:,i]==l)
        CX_Y.append(x_y)
    
    Py_x = np.zeros(k)
    for i in range(k):
        Py_x[i] = 1.
        for j in range(n):
            Py_x[i] = Py_x[i] * (CX_Y[j][i, x[j]] / CY[i])
        Py_x[i] = Py_x[i] * (CY[i] / N)
    ri = np.argmax(Py_x)
    return ri, Py_x[ri]

In [3]:
X = np.array([
    [0,0],
    [0,1],
    [0,1],
    [0,0],
    [0,0],
    [1,0],
    [1,1],
    [1,1],
    [1,2],
    [1,2],
    [2,2],
    [2,1],
    [2,1],
    [2,2],
    [2,2]
])
Y = np.array([
    [0],
    [0],
    [1],
    [1],
    [0],
    [0],
    [0],
    [1],
    [1],
    [1],
    [1],
    [1],
    [1],
    [1],
    [0]
])

In [4]:
naive_Bayes(np.array([1,0]), X, Y)

(0, 0.06666666666666667)

## 拉普拉斯平滑

In [5]:
def naive_Bayes_Laplace(x, X, Y, lam): ## 这里为了后面好写，把所有的值都统计了
    N, n = X.shape ## N样本数量，n属性个数
    k = np.max(Y) + 1
    Sx = np.max(X, axis=0) + 1
    CX_Y = [] ## 所有条件的计数
    CY = np.zeros(k) ## 所有先验的计数
    
    for i in range(k):
        CY[i] = len(np.where(Y[:,0]==i)[0])
    for i in range(n): ## 每个属性
        x_y = np.zeros((k, Sx[i]))
        for j in range(k): ## 每个y值
            t = X[np.where(Y[:,0]==j)[0]]
            for l in range(Sx[i]):
                x_y[j,l] = len(np.where(t[:,i]==l)[0])
        CX_Y.append(x_y)
    
    Py_x = np.zeros(k)
    for i in range(k):
        Py_x[i] = 1.
        for j in range(n):
            Py_x[i] = Py_x[i] * ((CX_Y[j][i, x[j]] + lam) / (CY[i] + Sx[j] * lam))
        Py_x[i] = Py_x[i] * ((CY[i] + lam) / (N + k * lam))
    ri = np.argmax(Py_x)
    return ri, Py_x[ri]

In [6]:
naive_Bayes_Laplace(np.array([1,0]), X, Y, 1.)

(0, 0.061002178649237467)

最后说下这个最小期望误差与后验最大化的关系

对于一个分类问题其期望误差为
$$
\begin{split}
R_{emp} & = E_{X\times Y}[L(Y,f(X))] \\
    & = \iint L(y, f(x)) p(y,x)\mathrm{d}y\mathrm{d}x \\
    & = \iint L(y, f(x)) p(y\mid x)\mathrm{d}y \cdot p(x)\mathrm{d}x \\
    & = \int \sum_c L(c, f(x))p(c\mid x) \cdot p(x)\mathrm{d}x \\
    & = E_X[\sum_c L(c, f(X))P(c\mid X)]
\end{split}
$$
故对应于每一个$x$，
$$
R_{emp}(x) = \sum_c L(c, f(x))P(c\mid x)
$$
由于对于分类问题
$$
L(c,f(x)) = \left\{\begin{matrix}
    1 & c \neq f(x)\\ 
    0 & c = f(x)
    \end{matrix}\right.
$$
故
$$
\begin{split}
f(x) & = \arg\underset{c}{\min} \sum_c L(c, f(x))P(c\mid x) \\
    & = \arg\underset{c}{\min} \sum_c P(y\neq c\mid x) \quad //f(x) = c \\
    & = \arg\underset{c}{\min} \Big(1 - P(y\neq c\mid x)\Big) \\
    & = \arg\underset{c}{\max} P(y = c \mid x)
\end{split}
$$